# How to handle tool errors

# Try/Except for Tool Calls
The simplest way to handle tool invocation errors is using a try/except block to catch and report errors.

In [5]:
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

# Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Define the tool
@tool
def complex_tool(int_arg: int, float_arg: float, dict_arg: dict) -> float:
    """Perform a calculation with the provided arguments."""
    return int_arg * float_arg + len(dict_arg)

# Bind tools to the LLM
llm_with_tools = llm.bind_tools([complex_tool])

# Error handling function
def try_except_tool(tool_args: dict, config: RunnableConfig):
    try:
        return complex_tool.invoke(tool_args, config=config)
    except Exception as e:
        return f"Tool invocation failed:\nArguments: {tool_args}\nError: {type(e)}: {e}"

# Example chain with error handling
chain = llm_with_tools | (lambda msg: {**msg.tool_calls[0]["args"], "dict_arg": {}}) | try_except_tool

# Test invocation
result = chain.invoke(
    "Use complex tool with args 5, 2.1, and an empty dictionary. Don't forget dict_arg."
)
print(result)


10.5


# . Fallback to a Better Model
If the tool call fails, you can retry using a fallback model.

In [6]:
# Define fallback model and chain
better_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0).bind_tools(
    [complex_tool], tool_choice="complex_tool"
)

better_chain = better_model | (lambda msg: msg.tool_calls[0]["args"]) | complex_tool

# Add a lambda function to provide default values for missing arguments
def handle_missing_args(args):
    # Provide a default empty dictionary for `dict_arg` if missing
    args.setdefault("dict_arg", {})
    return args

# Update the original chain with error handling for missing arguments
chain_with_error_handling = llm_with_tools | (lambda msg: handle_missing_args(msg.tool_calls[0]["args"])) | complex_tool

# Add fallback to the chain
chain_with_fallback = chain_with_error_handling.with_fallbacks([better_chain])

# Test invocation
result = chain_with_fallback.invoke(
    "Use complex tool with args 5, 2.1, and an empty dictionary. Don't forget dict_arg."
)
print(result)


10.5


# . Retry with Exception Details
For a more sophisticated error recovery, the chain can retry using the exception details to adjust its behavior.

In [4]:
from langchain_core.messages import AIMessage, HumanMessage, ToolCall, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

class CustomToolException(Exception):
    def __init__(self, tool_call: ToolCall, exception: Exception):
        super().__init__()
        self.tool_call = tool_call
        self.exception = exception

# Define tool custom exception handler
def tool_custom_exception(msg: AIMessage, config: RunnableConfig):
    try:
        return complex_tool.invoke(msg.tool_calls[0]["args"], config=config)
    except Exception as e:
        raise CustomToolException(msg.tool_calls[0], e)

# Convert exception to messages for retry
def exception_to_messages(inputs: dict):
    exception = inputs.pop("exception")
    messages = [
        AIMessage(content="", tool_calls=[exception.tool_call]),
        ToolMessage(
            tool_call_id=exception.tool_call["id"], content=str(exception.exception)
        ),
        HumanMessage(
            content="The last tool call raised an exception. Correct the arguments and retry."
        ),
    ]
    inputs["last_output"] = messages
    return inputs

# Prompt with placeholders for retries
prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}"), ("placeholder", "{last_output}")]
)

# Define chain with retry mechanism
chain = prompt | llm_with_tools | tool_custom_exception
self_correcting_chain = chain.with_fallbacks(
    [exception_to_messages | chain], exception_key="exception"
)

# Test invocation
result = self_correcting_chain.invoke(
    {"input": "Use complex tool with args 5, 2.1, and an empty dictionary. Don't forget dict_arg."}
)
print(result)


10.5
